# Load libraries and Network results

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Dec 01 11:41:42 2020

@author: sturkars
"""
import os
import sys
import glob
import numpy as np
import pandas as pd
#import miner as miner
from scipy.stats import zscore
import miner_py3_kk as miner
from tqdm.notebook import tqdm, trange
import time

# change the working directory
#os.chdir('/Users/serdarturkaslan/Documents/GitHub/GbmMINER/data/MINER_MicroLowessRNATMM.08.24.2020/')
# output directory
output_dir = "/Volumes/omics4tb2/SYGNAL/XCures"

# Path to the miner directory
input_path = "/Volumes/omics4tb2/SYGNAL/GBM-Serdar/MINER_MicroLowessRNATMM.08.24.2020"
print("Done")

Done


# Function to load regulons

In [2]:
def loadRegulons(disease_relevant=True,disease_relevant_regulons_file="regulonDfGbmMicroRNASigCoxAndStatSig.csv"):
    
    # Load regulon Modules
    regulonModules = miner.read_json(os.path.join(input_path,"regulons.json"))
    print("Total number of regulons: " + str(len(regulonModules)))

    # load regulon data frame
    # All Regulons
    regulonDf = pd.read_csv(os.path.join(input_path, "regulonDf.csv"), header = 0)
    regulonDf = list(regulonDf['Regulon_ID'].drop_duplicates())
    regulonDf = [str(i) for i in regulonDf]
    

    # Disease relevant regulons
    regulonDfMicroGbmLatest = pd.read_csv(os.path.join(input_path,disease_relevant_regulons_file), header = 0)
    regulonDfMicroGbmLatest = list(regulonDfMicroGbmLatest['Regulon_ID'].drop_duplicates())
    regulonDfMicroGbmLatest = [str(i) for i in regulonDfMicroGbmLatest]
    regulonModulesFiltered = dict((k, regulonModules[k]) for k in regulonDfMicroGbmLatest if k in regulonModules)
    print("Filtered number of regulons: " + str(len(regulonModulesFiltered)))
    
    if disease_relevant == True:
        #regulonModules = dict((k, regulonModules[k]) for k in regulonDfMicroGbmLatest if k in regulonModules)
        regulonModules = regulonModulesFiltered 
        print("Returned %s filtered regulons" %(str(len(regulonModules))))
        return(regulonModules)
    
    else:
        regulonModules = dict((k, regulonModules[k]) for k in regulonDf if k in regulonModules)
        print("Returned %s Total regulons" %(str(len(regulonModules))))
        return(regulonModules)


# Function to load Programs 

In [3]:
def loadPrograms(disease_relevant=True,disease_relevant_programs_file="transcriptional_programsmiRNAAndSig.json"):
    
    # Load transcriptional programs
    # All Programs
    transcriptional_programs = miner.read_json(os.path.join(input_path,'transcriptional_programs.json'))
    print("Total # of programs: ", str(len(transcriptional_programs)))
    
    transcriptional_programs_filtered = miner.read_json(os.path.join(input_path,disease_relevant_programs_file))
    print("Filtered # of programs: ", str(len(transcriptional_programs_filtered)))
    
    if disease_relevant == True:
            transcriptional_programs = transcriptional_programs_filtered
            print("Returned %s filtered Programs" %(str(len(transcriptional_programs_filtered))))
    else:
        transcriptional_programs = transcriptional_programs
        print("Returned %s Total Programs" %(str(len(transcriptional_programs))))

    program_list = [transcriptional_programs[str(key)] for key in range(len(transcriptional_programs.keys()))]
    
    return(program_list)

# Function to calculate Program activity

In [4]:
def calculateProgramActivity(program_list,regulons,expressionData,outputFile):
    
    # select reference dictionary for downstream analysis (pr_genes, revisedClusters, coexpressionModules, or regulonModules)
    referenceDictionary = createPrGenesDictionary(program_list,regulons)

    # create a background matrix used for statistical hypothesis testing
    bkgd = miner.backgroundDf(expressionData)

    # for each cluster, give samples that show high coherent cluster activity
    overExpressedMembers = miner.biclusterMembershipDictionary(referenceDictionary,bkgd,label=2,p=0.05)

    # for each clus|ter, give samples that show low coherent cluster activity
    underExpressedMembers = miner.biclusterMembershipDictionary(referenceDictionary,bkgd,label=0,p=0.05)

    # convert overExpressedMembers dictionary to binary matrix
    overExpressedProgramsMatrix = miner.membershipToIncidence(overExpressedMembers,expressionData)

    # convert underExpressedMembers dictionary to binary matrix
    underExpressedProgramsMatrix = miner.membershipToIncidence(underExpressedMembers,expressionData)

    # Create program matrix with values of {-1,0,1}
    dfr_programs = overExpressedProgramsMatrix-underExpressedProgramsMatrix
    
    # Write program activity into a outFile
    dfr_programs.to_csv(outputFile)
    
    return(dfr_programs)

# Function to create dictionary of program genes

In [5]:
# Create dictionary of program genes
# make dictionary of program keys with gene lists as elements
def createPrGenesDictionary(program_list, regulons):
    pr_genes_expanded = {}
    for i in range(len(program_list)):
        rgns = program_list[i]
        genes = []
        for r in rgns:
            if r in regulons:
                genes.append(regulons[r])
        if len(genes) != 0:
            genes = list(set(np.hstack(genes)))
            pr_genes_expanded[i] = genes

    unique_genes = set(x for y in pr_genes_expanded.values() for x in y)
        
    print("Total number of genes: " + str(len(unique_genes)))
    return(pr_genes_expanded)


# Load data folders

In [6]:
# Folders for patient data
patientDataFolders = glob.glob('/Volumes/omics4tb2/SYGNAL/XCures/P76156/*/RNA/results_RSEM/*.genes.results')
patientDataFolders = list(filter(lambda x:'P' in x, patientDataFolders))
print(patientDataFolders)
print('Total Patients to process: %s' %(len(patientDataFolders)))

# Folders for patient data for kallisto
# patientDataFolders = glob.glob('/Volumes/omics4tb2/SYGNAL/XCures/P76156/*/RNA/results_kallisto/abundance.tsv')
# patientDataFolders = list(filter(lambda x:'P' in x, patientDataFolders))
# print(patientDataFolders)
# print('Total Patients to process: %s' %(len(patientDataFolders)))

['/Volumes/omics4tb2/SYGNAL/XCures/P76156/P76156_3/RNA/results_RSEM/P76156_3.genes.results', '/Volumes/omics4tb2/SYGNAL/XCures/P76156/P76156_6/RNA/results_RSEM/P76156_6.genes.results']
Total Patients to process: 2


# Load Regulons

In [7]:
# load disease relevant regulons
regulonModulesDisease = loadRegulons(disease_relevant=True)

#load all regulons
regulonModulesAll = loadRegulons(disease_relevant=False)

Total number of regulons: 3764
Filtered number of regulons: 505
Returned 505 filtered regulons
Total number of regulons: 3764
Filtered number of regulons: 505
Returned 3764 Total regulons


# Load ribosomal genes

In [ ]:
# Load list of ribosomal proteisn to filter the expression data
ribosomal = pd.read_csv("../data/ribosomal_proteins.txt", header=0,index_col=0,sep="\t")
ribosomal = list(ribosomal["Ensembl gene ID"])

# load biomart gene/transcript mappings if using kallisto results
#biomart = pd.read_csv("../data/biomart_human_genes_mapping.txt", header=0,index_col=None,sep="\t")
#biomart = biomart[["Gene stable ID","Transcript stable ID version"]]
#biomart = biomart.rename({'Gene stable ID':'GeneID'}, axis=1)

# Load background genes

In [8]:
# We either use the MINER input expressioin genes as background or all propgram genes
def loadBackgroundGenes(backgd_type = "input"):
    if backgd_type == "input":
        # Use model input genes as background
        # Filter gene expression data only for genes that were in the model building data
        model_input_data = pd.read_csv(os.path.join(input_path,'GbmMicroRNAMergedWithIDsZScored.csv'),header=0,index_col=None,sep=",")
        model_input_genes = list(model_input_data['Unnamed: 0'])
        bkgd_genes = [item for sublist in model_input_genes for item in sublist]
        
        bkgd_genes = set(model_input_genes)
        print("background Genes:" + str(len(bkgd_genes)))
        bkgd_genes = [x for x in bkgd_genes if x not in ribosomal]
        print("background Genes after ribosomal remove:" + str(len(bkgd_genes)))
        return(bkgd_genes)
        
       
    if backgd_type == "program":
        # Use program genes as background
        # Filter gene expression data only for program genes across all programs
        program_list = loadPrograms(disease_relevant=False)
        referenceDictionary = createPrGenesDictionary(program_list, regulonModulesAll)
        my_genes =list(referenceDictionary.values())
        bkgd_genes = [item for sublist in my_genes for item in sublist]
        bkgd_genes = set(bkgd_genes)
        print("background Genes:" + str(len(bkgd_genes)))
        bkgd_genes = [x for x in bkgd_genes if x not in ribosomal]
        print("background Genes after ribosomal remove:" + str(len(bkgd_genes)))
        return(bkgd_genes)
    

# Load Programs

In [9]:
# load disease relevant programs
program_list_disease = loadPrograms(disease_relevant=True)

# load all programs
program_list_all = loadPrograms(disease_relevant=False)

Total # of programs:  178
Filtered # of programs:  33
Returned 33 filtered Programs
Total # of programs:  178
Filtered # of programs:  33
Returned 178 Total Programs


# Function to calculate activity stats

In [10]:
def activityStats(inputDf,label):
    # calculate regulon or program activity stats
    overActiveCount = len(inputDf[inputDf[newColName] == 1])
    underActiveCount = len(inputDf[inputDf[newColName] == -1])
    neutralCount = len(inputDf[inputDf[newColName] == 0])

    resSummary = {"Over" : [overActiveCount],
                          "Under" : [underActiveCount],
                          "Neutral" : [neutralCount],
                  "Type" : [label]
                 }
    resSummary = pd.DataFrame(resSummary)
    resSummary = resSummary.rename(index={0: patientID})
    return(resSummary)

# Loop through each patient to calculate Regulon & Program activity

In [16]:
# Load background genes
bkgd_genes = loadBackgroundGenes(backgd_type="input")

#import miner_py3_kk as miner
# Patient analysis loop
allSummary = pd.DataFrame()
for folder in tqdm(patientDataFolders):
    time.sleep(0.01)
    patientID = folder.split('/')[-1].split('.')[0] #get patientID
    print(patientID)
    #patientDataFile = os.path.join(folder,"RNA",patientID + ".genes.results") # get RNASeq results
    patientDataFile = folder
    
    # create patent activity output folder if it doesnt exist already
    activity_output_dir = os.path.join(output_dir,"patients_network_activities",patientID)
    
    if not os.path.isdir(activity_output_dir):
        os.mkdir(activity_output_dir)
    
    # Check if patient data file exists
    if os.path.exists('%s' %(patientDataFile)):
        # create a new column with patient name
        newColName = patientID + "_zscore"
        
        # Read expression data
        rawExpressionData = pd.read_csv(patientDataFile, sep="\t", index_col=None, header = 0)
        #print(rawExpressionData.head)
        
        # seperate ensembl gene ids and symbols
        rawExpressionData[['GeneID','gene_symbol']] = pd.DataFrame(rawExpressionData).gene_id.str.split("_",expand=True)
        #print(rawExpressionData)
        
        ## Merge etranscript IDs with Gene IDs for kallisto
        #rawExpressionData = rawExpressionData.merge(biomart, left_on='target_id', right_on='Transcript stable ID version', how='left')  
        
        # Filter ncRNAs or ribosomal RNAs
        rawExpressionData = rawExpressionData[-rawExpressionData.GeneID.isin(ribosomal)].copy()
        #print(rawExpressionData)
        
        # filter for genes in all programs
        #rawExpressionDataFilt = rawExpressionData[rawExpressionData.GeneID.isin(uniqueProteins)].copy()
        rawExpressionDataFilt = rawExpressionData[rawExpressionData.GeneID.isin(bkgd_genes)].copy()
        #rawExpressionDataFilt = rawExpressionData.loc[rawExpressionData.GeneID in my_genes]
        #rawExpressionDataFilt = rawExpressionData
        rawExpressionDataFilt.head()
        #print("Filtered raw expression")
        #print(rawExpressionDataFilt)
        
        # zscore patients expression data
        # kallisto
        #rawExpressionDataFilt[[newColName]] = rawExpressionDataFilt[['tpm']].apply(zscore)
        
        #START+RSEM
        rawExpressionDataFilt[[newColName]] = rawExpressionDataFilt[['TPM']].apply(zscore)
        rawExpressionDataFilt
        #expressionData = pd.read_csv("abundanceXCuresRNATPMTL1961DB85ZScoredLatest.csv", sep=",", index_col=0, header = 0)
        expressionData = rawExpressionDataFilt[[patientID + "_zscore"]]
        expressionData.index = rawExpressionDataFilt['GeneID']
        #print(expressionData)
        
        print("Processed: " + patientDataFile)
        # 
        ############### Regulon Activity ###############
        # calculate disease relevant regulon activity 
        rr = miner.generateRegulonActivity(regulonModulesDisease,expressionData, p=0.05)
        #print(rr)
        
        # calculate all regulon activity 
        aa = miner.generateRegulonActivity(regulonModulesAll,expressionData, p=0.05)
        #print(rr)
        
        #
        ############### Program Activity ###############
        # calculate disease relevant program activity
        program_activity_disease = calculateProgramActivity(program_list_disease,regulonModulesDisease,expressionData,outputFile= os.path.join(activity_output_dir,patientID) + "_disease_rel_program_activity.csv")
        #program_activity_disease = miner.generateProgramActivity(program_list_disease, regulonModulesDisease, expressionData, p=0.05, returnBkgd="no")
        
        # calculate all program activity
        program_activity_all = calculateProgramActivity(program_list_all,regulonModulesAll,expressionData,outputFile= os.path.join(activity_output_dir,patientID) + "_all_program_activity.csv")
    
        # write disease relevant regulon activity results to filr
        rr.to_csv(os.path.join(activity_output_dir,patientID) + "_disease_rel_regulon_activity.csv")
        
        # write all regulon activity results to file
        aa.to_csv(os.path.join(activity_output_dir,patientID) + "_all_regulon_activity.csv")
        
        # calculate regulon activity stats for disease relevant regulons
        
        sum1 = activityStats(rr,"Disease Relevant Regulons")
        sum2 = activityStats(aa,"All Regulons")
        sum3 = activityStats(program_activity_disease,"Disease Relevant Programs")
        sum4 = activityStats(program_activity_all,"All Programs")
        
        allSummary = pd.concat([allSummary,sum1,sum2,sum3,sum4])
    else:
        print("Data file doesnt exist: " + patientDataFile) 
print(allSummary)
        

background Genes:9728
background Genes after ribosomal remove:9614


  0%|          | 0/2 [00:00<?, ?it/s]

P76156_3
Processed: /Volumes/omics4tb2/SYGNAL/XCures/P76156/P76156_3/RNA/results_RSEM/P76156_3.genes.results
biclusterMembershipDictionary is done!
biclusterMembershipDictionary is done!
biclusterMembershipDictionary is done!
biclusterMembershipDictionary is done!
Total number of genes: 1862
biclusterMembershipDictionary is done!
biclusterMembershipDictionary is done!
Total number of genes: 6781
biclusterMembershipDictionary is done!
biclusterMembershipDictionary is done!
P76156_6
Processed: /Volumes/omics4tb2/SYGNAL/XCures/P76156/P76156_6/RNA/results_RSEM/P76156_6.genes.results
biclusterMembershipDictionary is done!
biclusterMembershipDictionary is done!
biclusterMembershipDictionary is done!
biclusterMembershipDictionary is done!
Total number of genes: 1862
biclusterMembershipDictionary is done!
biclusterMembershipDictionary is done!
Total number of genes: 6781
biclusterMembershipDictionary is done!
biclusterMembershipDictionary is done!
          Over  Under  Neutral                

## All Input genes

In [ ]:
background Genes:9728
background Genes after ribosomal remove:9614

Neutral  Over                       Type  Under
P76156_6      414    66  Disease Relevant Regulons     25
P76156_6     2691   808               All Regulons    265
P76156_6       22     7  Disease Relevant Programs      4
P76156_6      107    55               All Programs     16

# All program genes

In [ ]:
background Genes:6781
background Genes after ribosomal remove:6710

          Neutral  Over                       Type  Under
P76156_6      404    45  Disease Relevant Regulons     56
P76156_6     2743   640               All Regulons    381
P76156_6       19     4  Disease Relevant Programs     10
P76156_6      107    49               All Programs     22

In [68]:
expressionData.sort_values(by=["abundance_zscore"], ascending=False)

,abundance_zscore
GeneID,
ENSG00000133316,282.104016
ENSG00000197943,33.184662
ENSG00000120885,21.893781
ENSG00000161960,19.716571
ENSG00000135679,17.119100
...,...
ENSG00000282367,-0.043945
ENSG00000197858,-0.043945
ENSG00000282367,-0.043945


In [24]:
#rawExpressionData.sort_values(by=["P76156_6_zscore"], ascending=False)
zz1 = rawExpressionData.loc[-rawExpressionData['GeneID'].isin(ribosomal)].copy()
print(rawExpressionData.shape)
print(expressionData.shape)
print(zz1.shape)




(60508, 9)
(9228, 1)
(60508, 9)
